In [ ]:
'''
Chat GPT sugegstions on improvement - quite bad implimintation 
but maybe it is nice to separate functions "get deformations" and "perfom alignmetn"

It may be useful to break up the performAlignment method into smaller, more focused methods to make the code more 
readable and maintainable. Additionally, the class could benefit from some error handling, such as checking that the 
specified paths actually exist before attempting to open the geotiff images
'''  
import pathlib
import csv
import os
import time
import numpy as np
try:
    from osgeo import gdal
except:
    import gdal
from skimage.transform import PiecewiseAffineTransform, warp

from LocationMapping import LocationMapping


class Alignment:
    """Class for a pair of images alignment by displacements compensation"""

    def __init__(self, img1_path=None, img2_path=None, displacement_path=None,
                 transform_type="piecewise-affine", out_path=r'./',
                 transform_master=False, geocoded=False, proj=None):
        self.transform_type = transform_type
        self.img1_path = img1_path
        self.img2_path = img2_path
        self.displacement_path = displacement_path
        self.out_path = out_path
        self.transform_master = transform_master
        self.geocoded = geocoded
        self.proj = proj
        self.p = None
        self.image1 = None
        self.image2 = None
        self.displacements = None

        print(f'Alignment method: {transform_type}')
        os.makedirs(self.out_path, exist_ok=True)

        if not all((self.img1_path, self.img2_path, self.displacement_path)):
            raise TypeError("Please specify all necessary paths to data (Image 1, Image 2, and a path to displacement data)")

        # Check if the specified paths exist
        for path in (self.img1_path, self.img2_path, self.displacement_path):
            if not pathlib.Path(path).exists():
                raise FileNotFoundError(f"Specified file {path} does not exist")

        self.perform_alignment(padding='constant', back_alignment=False)

    def perform_alignment(self, padding, back_alignment):
        """
        Function for aligning two images given a deformation field and then save it to output path.
        """

        # Open first image and acquire raster as array
        self.image1 = gdal.Open(str(self.img1_path))

        # Open second image and acquire raster as array
        self.image2 = gdal.Open(str(self.img2_path))

        # Get deformation data as original pixel locations in image1 and new pixel locations in image1
        self.displacements = self.get_deformation_data()

        # Acquire transformation given by deformation
        self.get_transformation()

        # Save aligned image
        self.save_aligned_image(padding, back_alignment)

    def get_deformation_data(self):
        """
        Get displacement data and convert it to raster coordinates if it's in geocoded coordinates.
        """
        with open(self.displacement_path) as csvfile:
            csvreader = csv.reader(csvfile)
            has_header = csv.Sniffer().has_header(csvfile.read(1024))
            csvfile.seek(0)
            if has_header:
                next(csvreader)
            displacements = np.array([row for row in csvreader]).astype(float)
            if self.geocoded:k
                lm1 = LocationMapping(self.image1.GetGeoTransform(), self.image1.GetProjection())
                lm2 = LocationMapping(self.image2.GetGeoTransform(), self.image2.GetProjection())
                print('\nConverting geocoded coordinates to raster coordinates...')
                displacements = displacements[~np.any(np.isnan(displacements), axis=1), :]
                displacements = displacements[~np.any(np.isinf(displacements), axis=1), :]
                orig_locs = displacements[:, [0, 1]]
                new_locs = displacements[:, [2, 3]]
                c0, r0 = lm1.latLon2Raster(orig_locs[:, 1].reshape((-1)), orig_locs[:, 0].reshape((-1)))
                c1
